In [7]:
import config

In [8]:
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit, transpile

In [9]:
IBMProvider.save_account(token=config.IBMQ_TOKEN, overwrite=True)

In [10]:
circuit = QuantumCircuit(2, 2)

circuit.h(0)
circuit.cx(0, 1)
circuit.barrier()
circuit.measure(0, 0)
circuit.measure(1, 1)
circuit.draw()

┌───┐      ░ ┌─┐   
q_0: ┤ H ├──■───░─┤M├───
     └───┘┌─┴─┐ ░ └╥┘┌─┐
q_1: ─────┤ X ├─░──╫─┤M├
          └───┘ ░  ║ └╥┘
c: 2/══════════════╩══╩═
                   0  1

# 1. Run the circuit on a real quantum computer

In [11]:
provider = IBMProvider()

backend = provider.get_backend("ibm_osaka")

qc_basic = transpile(circuit, backend)
job = backend.run(qc_basic, shots=1024)
job_id = job.job_id()
print(job_id)

job = backend.retrieve_job(job_id)
result = job.result()
counts = result.get_counts()
print(counts)



crdmc60dqdn000835a8g


AttributeError: 'IBMBackend' object has no attribute 'retrieve_job'